# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Write functions to parse specific information from a web page
* Iterate over successive web pages in order to create a dataset

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
html_page = requests.get('http://books.toscrape.com/')
soup = BeautifulSoup(html_page.content, 'html.parser')

In [3]:
def retrieve_titles(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    titles = [h3.find('a').attrs['title'] for h3 in book_container.findAll('h3')]
    return titles
    

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given html page and return a list of the star-ratings for the books. These star ratings should be formatted as integers.

In [4]:
import re

In [5]:
def retrieve_ratings(soup):
    ratings = []
    regex = re.compile("star-rating (.*)")
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    for p in book_container.findAll('p', {"class": regex}):
        ratings.append(p.attrs['class'][-1])
    star_dict = {'One': 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five':5}
    ratings = [star_dict[s] for s in ratings]
    return ratings
    #Your code here

In [6]:
retrieve_ratings(soup)

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [7]:
def retrieve_prices(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    prices = [p.text for p in book_container.findAll('p', class_='price_color')]
    prices = [float(p[1:]) for p in prices]
    return prices
    
    



In [8]:
retrieve_prices(soup)

[51.77,
 53.74,
 50.1,
 47.82,
 54.23,
 22.65,
 33.34,
 17.93,
 22.6,
 52.15,
 13.99,
 20.66,
 17.46,
 52.29,
 35.02,
 57.25,
 23.88,
 37.59,
 51.33,
 45.17]

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [9]:
def retrieve_availabilities(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    availability = [p.text.strip() for p in book_container.findAll('p', class_="instock availability")]
    return availability
    
    
    #Your code here

In [10]:
retrieve_availabilities(soup)

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [44]:
import pandas as pd

df = pd.DataFrame()

In [45]:
#Your code here

dataframe_list = []

for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_availabilities(soup)
    df_i = pd.DataFrame([new_titles, new_star_ratings, new_prices, new_avails]).transpose()
    df = pd.concat([df, df_i])

df.columns = ['Title', 'Rating', 'Price', 'Availability']
df.reset_index()
    


,index,Title,Rating,Price,Availability
0,0,A Light in the Attic,3,51.77,In stock
1,1,Tipping the Velvet,1,53.74,In stock
2,2,Soumission,1,50.1,In stock
3,3,Sharp Objects,4,47.82,In stock
4,4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,5,The Requiem Red,1,22.65,In stock
6,6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,9,The Black Maria,1,52.15,In stock


In [46]:
df.drop(['index'], axis=1)
df

KeyError: "['index'] not found in axis"

## Level-Up: Write a new version of the script you just wrote. 

If you used url hacking to generate each successive page url, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!